## AWI SnowRadar (2-18 GHz) Snow Depth Example
The AWI radar has a larger bandwdith, improved range resolution , and is often operated at lower-altitude. This example uses the Peakiness algorithm developed to handle this dataset and derive snow depth.

Exmple AWI snow radar files are not available within this repo and must be requested.

In [ ]:
# pySnowRadar modules
from pySnowRadar import SnowRadar
from pySnowRadar.processing import extract_layers
from pySnowRadar.algorithms import Wavelet_TN, Peakiness

# Community packages
import os
import datetime
import time
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
radar_file = "../pySnowRadar/data/sr/Data_img_01_20190410_01_028.mat"
radar_dat = SnowRadar(radar_file, l_case='full')

In [ ]:
# Find surface and trim data
radar_dat.surf_bin, radar_dat.surface = radar_dat.get_surface()
radar_dat.plot_quicklook(ylim = list(radar_dat.get_bounds(5)))

## Application of wavelet and peakiness pickers

In [ ]:
params={'ref_snow_layer': 1,
        'cwt_precision': 10,
        'snow_density': 0.3,
        'log_peak_threshold': 0.6, 
        'lin_peak_threshold': 0.2, 
        'pp_l_threshold': 20, 
        'pp_r_threshold': 20}

In [ ]:
%%time
res_TN = extract_layers(radar_file, picker=Wavelet_TN, params=params, dump_results=False)

In [ ]:
%%time
res_Peakiness = extract_layers(radar_file, picker=Peakiness, params=params, dump_results=False)

In [ ]:
# Plotting the echogram with the picked interfaces and the resulting snow depth

fig, (ax1, ax2) = plt.subplots(2,1,figsize=(16,9),sharex=True)
ax1.imshow(10 * np.log10(radar_dat.data_radar/np.nanmax(radar_dat.data_radar)), interpolation='none', cmap="gist_gray")
ax1.plot(res_Peakiness['b_as'], '-', color='b',alpha=1/3,label='A-S peakiness')
ax1.plot(res_Peakiness['b_si'], '-', color='r',alpha=1/3,label='S-I peakiness')
ax1.set_ylabel("Range bin")
ax1.legend(loc="upper right")
ax1.set_ylim([np.nanmax(res_Peakiness['b_si'])+200, np.nanmin(res_Peakiness['b_as'])-400])
ax1.set_aspect('auto')

ax2.plot(res_Peakiness['snow_depth'],'.-',label='peakiness')
ax2.plot(res_TN['snow_depth'],':',color='C0', label='Haar')
ax2.axhline(res_Peakiness['snow_depth'].mean(),ls='--',color='k', label='avg')
ax2.set_ylabel("Snow depth [m]")
ax2.set_xlabel("Along track bin")
ax2.autoscale(enable=True, axis='x', tight=True)
ax2.legend(loc="upper right")
fig.tight_layout()